In [ ]:
class PuzzleSolver:
    def __init__(self, n, numlist, goal_state):
        self.n = n
        self.numlist = numlist
        self.expanded_states = [] # a unique list
        self.frontier = {} # a dict {state_n: [g(n), f(n)]}, desc by f(n) 
        self.goal_state = goal_state
        
    def get_zero(self, state):
        zero_id = state.index(0)
        return self.get_ij(zero_id)
    
    def get_ij(self, pos):
        return pos // self.n, pos % self.n
    
    def get_pos(self, i, j):
        return self.n * i + j
    
    def get_indv_manhattan(self, pos, val):
        goal_pos = self.goal_state.index(val)
        goal_i, goal_j = self.get_ij(goal_pos)
        
        pos_i, pos_j = self.get_ij(pos)
        
        indv_man_d = abs(goal_i + goal_j - pos_i - pos_j)
        
        return indv_man_d
    
    def get_hcost_manhattan(self, state):
        # get manhattan distance of all tile 
        # to each's correct position in goal state
        man_d = 0
        for i in range(self.n):
            man_d += self.get_indv_manhattan(i, state[i])
        return man_d
    
    def is_continue_searching(self):
        # state in frontier with least f
        next_state = None
        if len(self.frontier) > 0:
            next_state = list(self.frontier.items())[-1][0]
        return next_state != None and next_state != self.goal_state
    
    def swap_tile(self, curr_state, curr_i, curr_j, target_i, target_j):
        temp_state = curr_state.copy()
        tmp = temp_state[self.get_pos(curr_i, curr_j)]
        temp_state[curr_i, curr_j] = temp_state[target_i, target_j]
        temp_state[target_i, target_j] = tmp
        return temp_state
    
    def get_next_actions(self, curr_action):
        curr_state, curr_costs = curr_action
        curr_gcost, curr_fcost = curr_costs
        
        zero_i, zero_j = self.get_zero(curr_action)
        next_actions = []
        
        # top
        if zero_i > 0:
            top_i = zero_i - 1
            
            
        
        
    def solve(self):
        # get zero tile
        zero_i, zero_j = self.get_zero(self.numlist)
                
        # add input state into the frontier with current fcost
        # f(n) = g(n) + h(n)
        f_init_state = 0 + self.get_hcost_manhattan(self.numlist)
        self.frontier[self.numlist] = [0, f_init_state]
        
        # while there is still states in the frontier 
        # or not reaching goal state yet
        while self.is_continue_searching():
            curr_action = self.frontier.popitem() # move = (state, [g, f])
            
            # marked this state as visited
            self.expanded_states.append(curr_item[0])
            
            # generate a bunch of next actions
            

In [ ]:
1 4 7     2 5 7
2 5 8     1 4 8
3 6 0     0 3 6

In [ ]:
goal_i, goal_j = ij of goal.index(input_v[l = 8]: 6) = 2, 1
input_i, input_j = ij of 8 = 2, 2
man = abs(2+1-2-2) = 1

In [12]:
a = {'a': [6, 3], 'bb': [1, 16], 'aa': [3, 4], 'abc': [1,4,5]}
list(a.items())[-1]
ele = a.popitem()
print(ele)
c,d,f = ele[1]
print(c,d,f)

('abc', [1, 4, 5])
1 4 5


In [14]:
b = [1,2,3]
c = [1,2,3]
print(b == c)
d = c.copy()
d[0] = 100
c, d

True


([1, 2, 3], [100, 2, 3])